In [51]:
import os
import pandas as pd
import numpy as np
import nltk

In [2]:
def format_names():
    female_fn = '../names/female.txt'
    male_fn = '../names/male.txt'

    males = pd.read_csv(male_fn, skiprows=4)
    males.rename(columns={males.columns[0]:'MaleNames'}, inplace=True)

    females = pd.read_csv(female_fn, skiprows=4)
    females.rename(columns={females.columns[0]:'FemaleNames'}, inplace=True)

    merged = females.combine_first(males)
    merged = merged.applymap(lambda s:s.lower() if type(s) == str else s)
    
    return merged

In [71]:
def check_name_sex(name):
    name = name.lower()
    maleflags = merged['MaleNames'].eq(name).any()
    femaleflags = merged['FemaleNames'].eq(name).any()
    if maleflags and femaleflags:
        val = 'ambiguous'
    elif maleflags:
        val = 'male'
    elif femaleflags:
        val = 'female'
    else:
        val = 'unknown'
    print('The name {0} was decided to be {1}'.format(name, val))
    return val
        

In [94]:
x = ['john', 'mary', 'jean', 'ali', 'susan','mamta','harry','hermione','ron','peter','marvin','julie','jeremy','salvador']

In [95]:
for i in x:
    check_name_sex(i)

The name john was decided to be male
The name mary was decided to be female
The name jean was decided to be ambiguous
The name ali was decided to be ambiguous
The name susan was decided to be female
The name mamta was decided to be unknown
The name harry was decided to be male
The name hermione was decided to be female
The name ron was decided to be male
The name peter was decided to be male
The name marvin was decided to be male
The name julie was decided to be ambiguous
The name jeremy was decided to be male
The name salvador was decided to be male
